In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.interpolate import Akima1DInterpolator
#import matplotlib.pyplot as plt
#from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.decomposition import PCA
#from sklearn.model_selection import train_test_split

In [ ]:
pressure = pd.read_csv('pressure.csv')
wind = pd.read_csv('wind_speed.csv')
temperature = pd.read_csv('temperature.csv')
abs_humidity = pd.read_csv('abs_humidity.csv')
rel_humidity = pd.read_csv('rel_humidity.csv')
#altitudes = pd.read_csv('altitudes.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
df = pd.concat([wind[wind.columns],
                pressure[pressure.columns[1:]],
                abs_humidity[abs_humidity.columns[1:]],
                rel_humidity[abs_humidity.columns[1:]],
                temperature[temperature.columns[17:]]],
                      axis = 1)

In [ ]:
df2 = df.set_axis(['feature ' + str(i) for i in range(len(df.columns))], axis=1) 

In [ ]:
temperature["t"] = temperature["date"].apply(lambda x: datetime.fromisoformat(x).timestamp())
train["t"] = train["date"].apply(lambda x: datetime.fromisoformat(x).timestamp())
test["t"] = test["date"].apply(lambda x: datetime.fromisoformat(x).timestamp())

In [ ]:
inter_list = []

In [ ]:
t = temperature['t']

In [ ]:
df2['feature 0'] = df2['feature 0'].apply(lambda x: datetime.fromisoformat(x).timestamp())

In [ ]:
df2

In [ ]:
for item in df2.columns:
    inter_list.append(Akima1DInterpolator(t, df2[item]))

In [ ]:
len(inter_list)

In [ ]:
df2.rename(columns = {'feature 0':'date'},inplace = True)

In [ ]:
for i in range(len(inter_list)):
    train['feature ' + str(i)] = inter_list[i](train['t'])

In [ ]:
train.drop(columns = ['date','feature 0','id'],inplace = True)

In [ ]:
for i in range(len(inter_list)):
    test['feature ' + str(i)] = inter_list[i](test['t'])

In [ ]:
model1 = CatBoostRegressor(learning_rate = 0.02,
                           loss_function='RMSE',
                           task_type="CPU",
                           iterations = 15000,
                           l2_leaf_reg = 30,
                           random_state=42)
#best learning rate 0.02 i = 15000 leaf = 30

In [ ]:
model2 = CatBoostRegressor(learning_rate = 0.02,
                           loss_function='RMSE',
                           task_type="CPU",
                           iterations = 15000,
                           l2_leaf_reg = 30,
                           random_state = 42)
#l2 = 30, i = 15000, learning_rate = 0.1

In [ ]:
train

In [ ]:
model1.fit(train[train.columns[3:]],train['start'], verbose = False, plot = True)

#cat_t1.fit(start_df3[start_df3.columns[:-2]],start_df3['start'],verbose = False, plot = True)

In [ ]:
model2.fit(train[train.columns[3:]],train['end'],
           verbose = False,
           plot = True,
           )

In [ ]:
test

In [ ]:
#test.drop(columns = ['date','feature 0'], inplace = True)

In [ ]:
test['start'] = model1.predict(test[test.columns])
test['end'] = model2.predict(test[test.columns])

In [ ]:
test[["id", "start", "end"]].to_csv("submission.csv", index=False)